# Balance Sheet Display Demo

This notebook demonstrates the balance sheet visualization capabilities in bilancio.

**Important**: This notebook uses the actual agent classes (Bank, Household, etc.) because the policy engine checks class types for permissions.

In [1]:
# Import required modules
from bilancio.engines.system import System
from bilancio.domain.agent import Agent, AgentKind

# CRITICAL: Import actual agent classes - not just Agent with kind string!
from bilancio.domain.agents.bank import Bank
from bilancio.domain.agents.central_bank import CentralBank
from bilancio.domain.agents.household import Household
from bilancio.domain.agents.treasury import Treasury

from bilancio.analysis.balances import agent_balance, system_trial_balance
from bilancio.analysis.visualization import (
    display_agent_balance_table,
    display_agent_balance_from_balance,
    display_multiple_agent_balances
)
from bilancio.ops.banking import deposit_cash, withdraw_cash

## 1. Setup: Create a Financial System

We must use the actual agent classes because the policy engine checks `isinstance(agent, Bank)` etc.

In [2]:
# Initialize the system
system = System()

# Create the central bank - must use CentralBank class
central_bank = CentralBank(
    id="cb",
    name="Federal Reserve",
    kind="central_bank"  # Set automatically by CentralBank.__post_init__
)
system.bootstrap_cb(central_bank)

# Create other agents - must use proper classes!
agents = [
    Bank(
        id="bank1",
        name="Community Bank",
        kind="bank"  # Set automatically by Bank.__post_init__
    ),
    Household(
        id="hh1",
        name="The Smiths",
        kind="household"  # Set automatically by Household.__post_init__
    ),
    Household(
        id="hh2",
        name="The Johnsons",
        kind="household"
    ),
    Treasury(
        id="treasury",
        name="US Treasury",
        kind="treasury"  # Set automatically by Treasury.__post_init__
    )
]

# Add all agents at once using add_agents method
system.add_agents(agents)

print(f"System created with {len(system.state.agents)} agents:")
for agent_id, agent in system.state.agents.items():
    print(f"  - {agent.name} ({agent.__class__.__name__}, kind='{agent.kind}')")

System created with 5 agents:
  - Federal Reserve (CentralBank, kind='central_bank')
  - Community Bank (Bank, kind='bank')
  - The Smiths (Household, kind='household')
  - The Johnsons (Household, kind='household')
  - US Treasury (Treasury, kind='treasury')


## 2. Create Financial Relationships

In [3]:
# Central bank provides reserves to commercial bank
system.mint_reserves(to_bank_id="bank1", amount=10000)

# Issue cash to households
system.mint_cash(to_agent_id="hh1", amount=1000)
system.mint_cash(to_agent_id="hh2", amount=1500)

# Bank converts some reserves to cash (to have cash for withdrawals)
system.convert_reserves_to_cash(bank_id="bank1", amount=5000)

print("Initial financial relationships established")

Initial financial relationships established


In [4]:
# Households deposit cash at bank
deposit_cash(system, customer_id="hh1", bank_id="bank1", amount=800)
deposit_cash(system, customer_id="hh2", bank_id="bank1", amount=1200)

# Create a deliverable (non-financial asset)
system.create_deliverable(
    issuer_id="hh2",
    holder_id="hh1",
    sku="LAWN_SERVICE",
    quantity=1
)

print("Deposits and deliverables created")

Deposits and deliverables created


## 3. Display Single Agent Balance Sheet

In [5]:
# Display bank's balance sheet with rich formatting
print("Community Bank Balance Sheet (Rich Format):\n")
display_agent_balance_table(system, "bank1", format='rich')

Community Bank Balance Sheet (Rich Format):



                                 Community Bank (bank)                                 
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ cash                      │        7,000 │ bank_deposit              │        2,000 │
│ reserve_deposit           │        5,000 │                           │              │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │       12,000 │ TOTAL FINANCIAL           │        2,000 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │      +10,000 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

In [6]:
# Display the same with simple formatting
print("Community Bank Balance Sheet (Simple Format):\n")
display_agent_balance_table(system, "bank1", format='simple')

Community Bank Balance Sheet (Simple Format):


Community Bank (bank)
ASSETS                          Amount | LIABILITIES                     Amount
------------------------------------------------------------
cash                             7,000 | bank_deposit                     2,000
reserve_deposit                  5,000 |                                       
------------------------------------------------------------
TOTAL FINANCIAL                 12,000 | TOTAL FINANCIAL                  2,000
------------------------------------------------------------
                                        | NET FINANCIAL                  +10,000


In [7]:
# Display with custom title
print("Custom Title Example:\n")
display_agent_balance_table(
    system, 
    "bank1", 
    format='rich',
    title="Community Bank - Q4 2024"
)

Custom Title Example:



                               Community Bank - Q4 2024                                
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ cash                      │        7,000 │ bank_deposit              │        2,000 │
│ reserve_deposit           │        5,000 │                           │              │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │       12,000 │ TOTAL FINANCIAL           │        2,000 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │      +10,000 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

## 4. Display Balance Sheet from AgentBalance Object

In [8]:
# Get balance data for household 1
hh1_balance = agent_balance(system, "hh1")

# Analyze the data
print(f"The Smiths' Financial Position:")
print(f"  Total Financial Assets: ${hh1_balance.total_financial_assets:,}")
print(f"  Total Financial Liabilities: ${hh1_balance.total_financial_liabilities:,}")
print(f"  Net Financial Position: ${hh1_balance.net_financial:,}")
print(f"  Assets by kind: {dict(hh1_balance.assets_by_kind)}")
print(f"  Liabilities by kind: {dict(hh1_balance.liabilities_by_kind)}")
print()

# Display the balance sheet (no system parameter needed here!)
display_agent_balance_from_balance(hh1_balance, format='rich', title='The Smiths')

The Smiths' Financial Position:
  Total Financial Assets: $1,000
  Total Financial Liabilities: $0
  Net Financial Position: $1,000
  Assets by kind: {'cash': 200, 'bank_deposit': 800, 'deliverable': 1}
  Liabilities by kind: {}



                                      The Smiths                                       
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ bank_deposit              │          800 │                           │              │
│ cash                      │          200 │                           │              │
│ deliverable               │            1 │                           │              │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │        1,000 │ TOTAL FINANCIAL           │            0 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │       +1,000 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

## 5. Display Multiple Agent Balance Sheets Side by Side

Note: The correct parameter order is `(system, items, format)`

In [9]:
# Compare households
print("Household Comparison (Rich Format):\n")
display_multiple_agent_balances(system, ["hh1", "hh2"], format='rich')

Household Comparison (Rich Format):



          The Smiths                                                The Johnsons                                   
         (household)                                                (household)                                    
╭────────────────┬───────────╮                             ╭────────────────┬───────────╮                          
│ Item           │    Amount │                             │ Item           │    Amount │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ ASSETS         │           │                             │ ASSETS         │           │                          
│ bank_deposit   │       800 │                             │ bank_deposit   │     1,200 │                          
│ cash           │       200 │                             │ cash           │       300 │                          
│ deliverable    │         1 │                             │                │           │                          
│                │           │                             ├────────────────┼───────────┤                          
├────────────────┼───────────┤                             │ LIABILITIES    │           │                          
│ LIABILITIES    │           │                             │ deliverable    │         1 │                          
│                │           │                             │                │           │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ Total          │     1,000 │                             │ Total          │     1,500 │                          
│ Financial      │           │                             │ Financial      │           │                          
│ Total Liab.    │         0 │                             │ Total Liab.    │         0 │                          
│ Net Financial  │    +1,000 │                             │ Net Financial  │    +1,500 │                          
╰────────────────┴───────────╯                             ╰────────────────┴───────────╯

In [10]:
# Compare banking system
print("Banking System (Rich Format):\n")
display_multiple_agent_balances(system, ["cb", "bank1"], format='rich')

Banking System (Rich Format):



       Federal Reserve                                             Community Bank                                  
        (central_bank)                                                 (bank)                                      
╭────────────────┬───────────╮                             ╭────────────────┬───────────╮                          
│ Item           │    Amount │                             │ Item           │    Amount │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ ASSETS         │           │                             │ ASSETS         │           │                          
│                │           │                             │ cash           │     7,000 │                          
├────────────────┼───────────┤                             │ reserve_depos… │     5,000 │                          
│ LIABILITIES    │           │                             │                │           │                          
│ cash           │     7,500 │                             ├────────────────┼───────────┤                          
│ reserve_depos… │     5,000 │                             │ LIABILITIES    │           │                          
│                │           │                             │ bank_deposit   │     2,000 │                          
├────────────────┼───────────┤                             │                │           │                          
│ Total          │         0 │                             ├────────────────┼───────────┤                          
│ Financial      │           │                             │ Total          │    12,000 │                          
│ Total Liab.    │    12,500 │                             │ Financial      │           │                          
│ Net Financial  │   -12,500 │                             │ Total Liab.    │     2,000 │                          
╰────────────────┴───────────╯                             │ Net Financial  │   +10,000 │                          
                                                           ╰────────────────┴───────────╯

In [11]:
# Simple format comparison
print("Banking System (Simple Format):\n")
display_multiple_agent_balances(system, ["cb", "bank1"], format='simple')

Banking System (Simple Format):


              Federal Reserve (central_bank)               |                   Community Bank (bank)                   
---------------------------------------------------------- | ----------------------------------------------------------
ASSETS                                                     | ASSETS                                                    
                                                           |                                                 cash 7,000
LIABILITIES                                                |                                      reserve_deposit 5,000
                                                cash 7,500 |                                                           
                                     reserve_deposit 5,000 | LIABILITIES                                               
                                                           |                                         bank_deposit 2,000
      

In [12]:
# Mix agent IDs and AgentBalance objects
cb_balance = agent_balance(system, "cb")
bank_balance = agent_balance(system, "bank1")

print("Mixed Input Types:\n")
display_multiple_agent_balances(system, [cb_balance, "hh1", bank_balance, "hh2"], format='rich')

Mixed Input Types:



       Federal Reserve                           The Smiths                           Community Bank               
        (central_bank)                          (household)                               (bank)                   
╭────────────────┬───────────╮         ╭────────────────┬───────────╮         ╭────────────────┬───────────╮       
│ Item           │    Amount │         │ Item           │    Amount │         │ Item           │    Amount │       
├────────────────┼───────────┤         ├────────────────┼───────────┤         ├────────────────┼───────────┤       
│ ASSETS         │           │         │ ASSETS         │           │         │ ASSETS         │           │       
│                │           │         │ bank_deposit   │       800 │         │ cash           │     7,000 │       
├────────────────┼───────────┤         │ cash           │       200 │         │ reserve_depos… │     5,000 │       
│ LIABILITIES    │           │         │ deliverable    │         1 │         │                │           │       
│ cash           │     7,500 │         │                │           │         ├────────────────┼───────────┤       
│ reserve_depos… │     5,000 │         ├────────────────┼───────────┤         │ LIABILITIES    │           │       
│                │           │         │ LIABILITIES    │           │         │ bank_deposit   │     2,000 │       
├────────────────┼───────────┤         │                │           │         │                │           │       
│ Total          │         0 │         ├────────────────┼───────────┤         ├────────────────┼───────────┤       
│ Financial      │           │         │ Total          │     1,000 │         │ Total          │    12,000 │       
│ Total Liab.    │    12,500 │         │ Financial      │           │         │ Financial      │           │       
│ Net Financial  │   -12,500 │         │ Total Liab.    │         0 │         │ Total Liab.    │     2,000 │       
╰────────────────┴───────────╯         │ Net Financial  │    +1,000 │         │ Net Financial  │   +10,000 │       
                                       ╰────────────────┴───────────╯         ╰────────────────┴───────────╯       
         The Johnsons                                                                                              
         (household)                                                                                               
╭────────────────┬───────────╮                                                                                     
│ Item           │    Amount │                                                                                     
├────────────────┼───────────┤                                                                                     
│ ASSETS         │           │                                                                                     
│ bank_deposit   │     1,200 │                                                                                     
│ cash           │       300 │                                                                                     
│                │           │                                                                                     
├────────────────┼───────────┤                                                                                     
│ LIABILITIES    │           │                                                                                     
│ deliverable    │         1 │                                                                                     
│                │           │                                                                                     
├────────────────┼───────────┤                                                                                     
│ Total          │     1,500 │                                                                                     
│ Financial      │           │                          

In [13]:
# Display all agents
print("Complete System Overview:\n")
all_agent_ids = list(system.state.agents.keys())
display_multiple_agent_balances(system, all_agent_ids, format='simple')

Complete System Overview:


Federal Reserve (centr... |   Community Bank (bank)   |   The Smiths (household)  |  The Johnsons (household) |   US Treasury (treasury) 
------------------------- | ------------------------- | ------------------------- | ------------------------- | -------------------------
ASSETS                    | ASSETS                    | ASSETS                    | ASSETS                    | ASSETS                   
                          |                cash 7,000 |          bank_deposit 800 |        bank_deposit 1,200 |                          
LIABILITIES               |     reserve_deposit 5,000 |                  cash 200 |                  cash 300 | LIABILITIES              
               cash 7,500 |                           |             deliverable 1 |                           |                          
    reserve_deposit 5,000 | LIABILITIES               |                           | LIABILITIES               |         Total Financial 0
      

## 6. System-Wide Trial Balance

In [14]:
# Get system trial balance
trial_balance = system_trial_balance(system)

print("System-Wide Trial Balance:")
print("="*50)
print("\nAssets by Kind:")
for kind, amount in sorted(trial_balance.assets_by_kind.items()):
    print(f"  {kind:20} ${amount:,}")

print("\nLiabilities by Kind:")
for kind, amount in sorted(trial_balance.liabilities_by_kind.items()):
    print(f"  {kind:20} ${amount:,}")

print("\nTotals:")
print(f"  Total Financial Assets:      ${trial_balance.total_financial_assets:,}")
print(f"  Total Financial Liabilities: ${trial_balance.total_financial_liabilities:,}")

# Verify double-entry bookkeeping
if trial_balance.total_financial_assets == trial_balance.total_financial_liabilities:
    print("\n✅ System is in balance (Assets = Liabilities)")
else:
    print("\n❌ System is NOT in balance!")
    print(f"   Difference: ${abs(trial_balance.total_financial_assets - trial_balance.total_financial_liabilities):,}")

System-Wide Trial Balance:

Assets by Kind:
  bank_deposit         $2,000
  cash                 $7,500
  deliverable          $1
  reserve_deposit      $5,000

Liabilities by Kind:
  bank_deposit         $2,000
  cash                 $7,500
  deliverable          $1
  reserve_deposit      $5,000

Totals:
  Total Financial Assets:      $14,500
  Total Financial Liabilities: $14,500

✅ System is in balance (Assets = Liabilities)


## 7. Transaction Examples

In [15]:
# Cash withdrawal
print("Before withdrawal:\n")
display_multiple_agent_balances(system, ["hh1", "bank1"], format='rich')

# Withdraw cash
withdraw_cash(system, customer_id="hh1", bank_id="bank1", amount=300)

print("\n" + "="*60)
print("\nAfter $300 withdrawal:\n")
display_multiple_agent_balances(system, ["hh1", "bank1"], format='rich')

Before withdrawal:



          The Smiths                                               Community Bank                                  
         (household)                                                   (bank)                                      
╭────────────────┬───────────╮                             ╭────────────────┬───────────╮                          
│ Item           │    Amount │                             │ Item           │    Amount │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ ASSETS         │           │                             │ ASSETS         │           │                          
│ bank_deposit   │       800 │                             │ cash           │     7,000 │                          
│ cash           │       200 │                             │ reserve_depos… │     5,000 │                          
│ deliverable    │         1 │                             │                │           │                          
│                │           │                             ├────────────────┼───────────┤                          
├────────────────┼───────────┤                             │ LIABILITIES    │           │                          
│ LIABILITIES    │           │                             │ bank_deposit   │     2,000 │                          
│                │           │                             │                │           │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ Total          │     1,000 │                             │ Total          │    12,000 │                          
│ Financial      │           │                             │ Financial      │           │                          
│ Total Liab.    │         0 │                             │ Total Liab.    │     2,000 │                          
│ Net Financial  │    +1,000 │                             │ Net Financial  │   +10,000 │                          
╰────────────────┴───────────╯                             ╰────────────────┴───────────╯



After $300 withdrawal:



          The Smiths                                               Community Bank                                  
         (household)                                                   (bank)                                      
╭────────────────┬───────────╮                             ╭────────────────┬───────────╮                          
│ Item           │    Amount │                             │ Item           │    Amount │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ ASSETS         │           │                             │ ASSETS         │           │                          
│ bank_deposit   │       500 │                             │ cash           │     6,700 │                          
│ cash           │       500 │                             │ reserve_depos… │     5,000 │                          
│ deliverable    │         1 │                             │                │           │                          
│                │           │                             ├────────────────┼───────────┤                          
├────────────────┼───────────┤                             │ LIABILITIES    │           │                          
│ LIABILITIES    │           │                             │ bank_deposit   │     1,700 │                          
│                │           │                             │                │           │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ Total          │     1,000 │                             │ Total          │    11,700 │                          
│ Financial      │           │                             │ Financial      │           │                          
│ Total Liab.    │         0 │                             │ Total Liab.    │     1,700 │                          
│ Net Financial  │    +1,000 │                             │ Net Financial  │   +10,000 │                          
╰────────────────┴───────────╯                             ╰────────────────┴───────────╯

In [16]:
# Cash transfer between households
print("Cash transfer: hh1 pays hh2 $100\n")
print("Before:\n")
display_multiple_agent_balances(system, ["hh1", "hh2"], format='rich')

# Transfer cash
system.transfer_cash(from_agent_id="hh1", to_agent_id="hh2", amount=100)

print("\n" + "="*60)
print("\nAfter transfer:\n")
display_multiple_agent_balances(system, ["hh1", "hh2"], format='rich')

Cash transfer: hh1 pays hh2 $100

Before:



          The Smiths                                                The Johnsons                                   
         (household)                                                (household)                                    
╭────────────────┬───────────╮                             ╭────────────────┬───────────╮                          
│ Item           │    Amount │                             │ Item           │    Amount │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ ASSETS         │           │                             │ ASSETS         │           │                          
│ bank_deposit   │       500 │                             │ bank_deposit   │     1,200 │                          
│ cash           │       500 │                             │ cash           │       300 │                          
│ deliverable    │         1 │                             │                │           │                          
│                │           │                             ├────────────────┼───────────┤                          
├────────────────┼───────────┤                             │ LIABILITIES    │           │                          
│ LIABILITIES    │           │                             │ deliverable    │         1 │                          
│                │           │                             │                │           │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ Total          │     1,000 │                             │ Total          │     1,500 │                          
│ Financial      │           │                             │ Financial      │           │                          
│ Total Liab.    │         0 │                             │ Total Liab.    │         0 │                          
│ Net Financial  │    +1,000 │                             │ Net Financial  │    +1,500 │                          
╰────────────────┴───────────╯                             ╰────────────────┴───────────╯



After transfer:



          The Smiths                                                The Johnsons                                   
         (household)                                                (household)                                    
╭────────────────┬───────────╮                             ╭────────────────┬───────────╮                          
│ Item           │    Amount │                             │ Item           │    Amount │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ ASSETS         │           │                             │ ASSETS         │           │                          
│ bank_deposit   │       500 │                             │ bank_deposit   │     1,200 │                          
│ cash           │       400 │                             │ cash           │       400 │                          
│ deliverable    │         1 │                             │                │           │                          
│                │           │                             ├────────────────┼───────────┤                          
├────────────────┼───────────┤                             │ LIABILITIES    │           │                          
│ LIABILITIES    │           │                             │ deliverable    │         1 │                          
│                │           │                             │                │           │                          
├────────────────┼───────────┤                             ├────────────────┼───────────┤                          
│ Total          │       900 │                             │ Total          │     1,600 │                          
│ Financial      │           │                             │ Financial      │           │                          
│ Total Liab.    │         0 │                             │ Total Liab.    │         0 │                          
│ Net Financial  │      +900 │                             │ Net Financial  │    +1,600 │                          
╰────────────────┴───────────╯                             ╰────────────────┴───────────╯

## 8. Display All Agents

In [17]:
# Display each agent's balance sheet
print("Complete System State:\n")

for agent_id in system.state.agents.keys():
    agent = system.state.agents[agent_id]
    print(f"\n{'='*60}")
    display_agent_balance_table(
        system, 
        agent_id, 
        format='rich',
        title=f"{agent.name} ({agent.__class__.__name__})"
    )

Complete System State:




                             Federal Reserve (CentralBank)                             
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ cash                      │        7,500 │
│                           │              │ reserve_deposit           │        5,000 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │            0 │ TOTAL FINANCIAL           │       12,500 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │      -12,500 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

                                 Community Bank (Bank)                                 
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ cash                      │        6,700 │ bank_deposit              │        1,700 │
│ reserve_deposit           │        5,000 │                           │              │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │       11,700 │ TOTAL FINANCIAL           │        1,700 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │      +10,000 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

                                The Smiths (Household)                                 
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ bank_deposit              │          500 │                           │              │
│ cash                      │          400 │                           │              │
│ deliverable               │            1 │                           │              │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │          900 │ TOTAL FINANCIAL           │            0 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │         +900 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

                               The Johnsons (Household)                                
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ bank_deposit              │        1,200 │ deliverable               │            1 │
│ cash                      │          400 │                           │              │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │        1,600 │ TOTAL FINANCIAL           │            0 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │       +1,600 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

                                US Treasury (Treasury)                                 
╭───────────────────────────┬──────────────┬───────────────────────────┬──────────────╮
│ ASSETS                    │       Amount │ LIABILITIES               │       Amount │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │                           │              │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│ TOTAL FINANCIAL           │            0 │ TOTAL FINANCIAL           │            0 │
│                           │              │                           │              │
├───────────────────────────┼──────────────┼───────────────────────────┼──────────────┤
│                           │              │ NET FINANCIAL             │            0 │
╰───────────────────────────┴──────────────┴───────────────────────────┴──────────────╯

## Summary

### Key Lessons Learned:

1. **Must use proper agent classes** (`Bank`, `Household`, etc.) not just `Agent` with kind string
   - The policy engine checks `isinstance(agent, Bank)` for permissions
   - Using `Agent(kind="bank")` will fail validation

2. **Correct API usage**:
   - `display_multiple_agent_balances(system, items, format)` - system first!
   - `display_agent_balance_from_balance(balance, format, title)` - no system param

3. **AgentKind enum** exists but classes set their own kind in `__post_init__`

### Visualization Functions:
- `display_agent_balance_table()` - Single agent balance sheet
- `display_agent_balance_from_balance()` - From AgentBalance object
- `display_multiple_agent_balances()` - Compare multiple agents

### Features Demonstrated:
- Rich vs Simple formatting
- T-account style presentation
- Custom titles
- System-wide trial balance
- Transaction examples with before/after comparisons